但现在我们需要制作了一个新的csv：df = pd.read_csv('/root/Download/AlgaeBloomForecast/merged_data.csv')

```
date,temp,oxygen,NH3,TP,TN,algae,area,weather,max_temperature,min_temperature,aqi,aqiLevel,wind_direction,wind_power,aqiInfo
2021-06-02,26.1875,6.6665,0.025,0.068275,1.07325,14400000.0,无锡,阴-阵雨,26,21,24,1,东南风,4级,优
2021-06-03,25.881666666666664,6.6418333333333335,0.0251166666666666,0.0637833333333333,0.9151666666666666,10867091.666666666,无锡,阴-阵雨,26,19,66,2,西北风,3级,良
2021-06-04,25.895,7.946333333333333,0.025,0.0637833333333333,0.9203333333333332,25498423.33333333,无锡,阴-多云,26,18,51,2,西南风,3级,良
2021-06-05,26.85,9.084,0.025,0.04776,0.9058,21100000.0,无锡,晴,32,19,67,2,西南风,3级,良
2021-06-06,28.256666666666664,9.514333333333331,0.025,0.0440666666666666,0.9233333333333332,15211340.0,无锡,晴,33,19,80,2,南风,3级,良
2021-06-07,27.635,8.3865,0.025,0.0366499999999999,0.7778333333333333,7994458.333333333,无锡,阴-多云,35,21,68,2,东南风,3级,良
2021-06-08,28.19666666666667,8.397499999999999,0.025,0.0418666666666666,0.7323333333333334,12259158.333333334,无锡,阴-多云,30,24,36,1,东南风,3级,优
2021-06-09,28.751666666666665,8.309166666666668,0.025,0.0389833333333333,0.601,6891956.666666667,无锡,阴-雷阵雨,32,24,52,2,东南风,3级,良
2021-06-10,28.741666666666664,7.385833333333333,0.025,0.03785,0.5256666666666666,6301236.666666667,无锡,阴,28,24,38,1,东南风,2级,优
2021-06-11,29.491666666666664,7.6176666666666675,0.025,0.0327666666666666,0.4495,6244151.666666667,无锡,阴-多云,32,23,82,2,东风,2级,良
2021-06-12,29.58666666666667,7.271999999999999,0.025,0.02975,0.3741666666666667,4201731.666666667,无锡,多云-雷阵雨,33,24,41,1,东南风,3级,优
2021-06-13,29.563333333333333,6.929333333333333,0.025,0.0302833333333333,0.2663333333333333,4964940.0,无锡,阴-小雨,28,25,34,1,西南风,2级,优
2021-06-14,29.58833333333333,6.963166666666666,0.025,0.0290666666666666,0.1886666666666666,5394340.0,无锡,阴-小雨,31,25,46,1,东南风,3级,优
2021-06-15,30.21,7.23925,0.025,0.033425,0.396,6927237.5,无锡,阴-小雨,33,24,48,1,西南风,3级,优
```

这里我们要用1D CNN预测藻类的爆发，来捕获时间序列数据的长距离依赖关系。这里提供了逐日的数据，需要考虑date的影响。

通过前期的数据分析，我们发现：
- 这里需要考虑['temp', 'oxygen', 'NH3', 'TP', 'TN'，'algae']对'algae'的影响
- 这里需要考虑weather这一列出现“晴”的影响，以及“晴”的长距离依赖关系。
- 温度的影响，这里，只采用temp作为特征，考虑温度的长距离依赖关系。
- 不考虑max_temperature,min_temperature。
- wind_power大于4级时是一个需要考虑的因素，考虑wind_power的长距离依赖关系。
- 这里的aqi,aqiLevel，aqiInfo与空气质量相关，不考虑。
- wind_direction也不考虑。

请你输出完整的代码。需要：
- 做归一化的处理
- 需要对数据分割，将最近180天的结果作为测试集。
- 需要绘制训练过程的图片、需要输出预测效果的图片，它将生成两个图像文件：`training_loss_1D_CNN.png`显示训练过程，`prediction_results_1D_CNN.png`显示预测效果
- 模型的训练参数是可调整的，比如，添加了学习率调度（ReduceLROnPlateau）、在优化器中添加了L2正则化（weight_decay）、调整了模型结构，包括隐藏层维度和dropout率。

1D CNN、TCN和LSTM这三种模型在处理序列数据时确实存在一些本质区别。让我们逐一比较:

1. 1D CNN (一维卷积神经网络):

- 结构: 使用一维卷积核沿时间维度滑动。
- 特点: 能够捕捉局部模式和特征，但感受野有限。
- 并行性: 高度并行，训练和推理速度快。
- 长期依赖: 难以直接捕捉长期依赖关系。

2. TCN (时间卷积网络):

- 结构: 基于1D CNN，但使用膨胀卷积和残差连接。
- 特点: 可以捕捉更长范围的时间依赖。
- 并行性: 保持了CNN的并行性优势。
- 长期依赖: 通过膨胀卷积可以有效处理长期依赖。

3. LSTM (长短期记忆网络):

- 结构: 循环结构，包含门控机制和内部状态。
- 特点: 能够学习长期依赖关系。
- 并行性: 序列处理，难以并行化。
- 长期依赖: 专门设计用于处理长期依赖问题。

本质区别:

1. 架构差异:
   - 1D CNN和TCN是前馈网络，而LSTM是循环网络。
   - 1D CNN和TCN使用卷积操作，LSTM使用门控机制。

2. 时间依赖处理:
   - 1D CNN主要捕捉局部时间特征。
   - TCN通过膨胀卷积扩大感受野，可以处理更长范围的依赖。
   - LSTM通过内部状态和门控机制显式地处理长期依赖。

3. 并行性和计算效率:
   - 1D CNN和TCN可以高度并行化，计算效率高。
   - LSTM由于其循环性质，难以并行化，但在某些序列任务中可能更有效。

4. 内存使用:
   - 1D CNN和TCN的内存使用与输入序列长度成正比。
   - LSTM可以通过内部状态压缩长序列信息，潜在的内存效率更高。

5. 梯度流:
   - 1D CNN和TCN没有梯度消失/爆炸问题。
   - LSTM通过门控机制缓解了传统RNN的梯度问题，但仍可能在极长序列中遇到困难。

总的来说，这三种模型在处理时序数据时各有优势。选择哪种模型通常取决于具体的应用场景、数据特性和计算资源限制。

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 读取数据
df = pd.read_csv('/root/Download/AlgaeBloomForecast/merged_data.csv')

# 数据预处理
df['date'] = pd.to_datetime(df['date'])
df['is_sunny'] = df['weather'].str.contains('晴').astype(int)
df['high_wind'] = (df['wind_power'].str.extract('(\d+)').astype(float) > 4).astype(int)

# 选择特征
features = ['temp', 'oxygen', 'NH3', 'TP', 'TN', 'is_sunny', 'high_wind']
target = 'algae'

# 分别对特征和目标进行归一化
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

df[features] = scaler_X.fit_transform(df[features])
df[[target]] = scaler_y.fit_transform(df[[target]])

# 准备序列数据
sequence_length = 30  # 使用过去30天的数据来预测

X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i+sequence_length].values)
    y.append(df[target].iloc[i+sequence_length])

X = np.array(X)
y = np.array(y)

# 分割数据集，最后180天作为测试集
test_size = 180
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

# 转换为PyTorch张量
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# 创建数据加载器
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_30762/3461968803.py:17: SyntaxWarning: invalid escape sequence '\d'
  df['high_wind'] = (df['wind_power'].str.extract('(\d+)').astype(float) > 4).astype(int)


In [ ]:


# 定义1D CNN模型
class AlgaeBloomCNN(nn.Module):
    def __init__(self):
        super(AlgaeBloomCNN, self).__init__()
        self.conv1 = nn.Conv1d(7, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.4)
        self.flatten = nn.Flatten()
        
        # 计算全连接层的输入维度
        self.fc1_input_dim = self._get_conv_output_dim()
        
        self.fc1 = nn.Linear(self.fc1_input_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def _get_conv_output_dim(self):
        # 创建一个虚拟输入来计算卷积层的输出维度
        dummy_input = torch.zeros(1, 7, 30)
        x = self.pool(self.conv1(dummy_input))
        x = self.pool(self.conv2(x))
        x = self.flatten(x)
        return x.shape[1]
        
    def forward(self, x):
        x = x.permute(0, 2, 1)  # 调整输入维度为 (batch_size, channels, sequence_length)
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout2(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze()


In [4]:

# 初始化模型、损失函数和优化器
model = AlgaeBloomCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

In [5]:


# 训练模型
num_epochs = 200
train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    scheduler.step(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 绘制训练过程
plt.figure(figsize=(10, 5))
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig('training_loss_1D_CNN.png')
plt.close()


# 预测
model.eval()
with torch.no_grad():
    y_pred = model(X_test).numpy()
    y_true = y_test.numpy()

# 反归一化
y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))[:, 0]
y_true = scaler_y.inverse_transform(y_true.reshape(-1, 1))[:, 0]

# 绘制预测结果
plt.figure(figsize=(12, 6))
plt.plot(df['date'].iloc[-test_size:], y_true, label='Actual')
plt.plot(df['date'].iloc[-test_size:], y_pred, label='Predicted')
plt.title('Algae Bloom Prediction')
plt.xlabel('Date')
plt.ylabel('Algae Concentration')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('prediction_results_1D_CNN.png')
plt.close()

# 计算评估指标
mse = np.mean((y_true - y_pred)**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_true - y_pred))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R-squared: {r2:.4f}')

Epoch [10/200], Loss: 0.0044
Epoch [20/200], Loss: 0.0041
Epoch [30/200], Loss: 0.0028
Epoch [40/200], Loss: 0.0023
Epoch [50/200], Loss: 0.0019
Epoch [60/200], Loss: 0.0018
Epoch [70/200], Loss: 0.0015
Epoch [80/200], Loss: 0.0015
Epoch [90/200], Loss: 0.0011
Epoch [100/200], Loss: 0.0011
Epoch [110/200], Loss: 0.0010
Epoch [120/200], Loss: 0.0009
Epoch [130/200], Loss: 0.0008
Epoch [140/200], Loss: 0.0009
Epoch [150/200], Loss: 0.0007
Epoch [160/200], Loss: 0.0006
Epoch [170/200], Loss: 0.0006
Epoch [180/200], Loss: 0.0006
Epoch [190/200], Loss: 0.0005
Epoch [200/200], Loss: 0.0006
Mean Squared Error: 1047706403340288.0000
Root Mean Squared Error: 32368294.0000
Mean Absolute Error: 20921546.0000
R-squared: -0.1731
